In [27]:
import json
from openai import OpenAI
import pandas as pd

/var/folders/br/hdgw7t_11777tltdjs9tcsnc0000gn/T/ipykernel_34851/3216677182.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [71]:
with open('../data/recipes.json') as file:
    recipes_data = json.load(file)

In [72]:
[recipe['description'] for recipe in recipes_data.values() if recipe.get('description') is not None]

['This grain bowl combines roasted mushrooms, halloumi cheese, and barley with fresh vegetables, herbs, and a tangy yogurt-olive sauce for a satisfying and flavorful meal. Perfect for dinner or lunch.',
 "This vibrant roasted squash and radicchio salad is tossed in a tangy buttermilk dressing. It's a delicious and light side dish, perfect for showcasing the flavors of the season.",
 'These Rice Krispie treats are made with browned butter, giving them a rich and nutty flavor. Topped with rainbow sprinkles, they are a fun and nostalgic snack or dessert option. So easy to make, and always a crowd-pleaser!',
 'This easy and quick tofu and tomato egg drop soup is perfect for a weeknight meal. The combination of tangy tomatoes, silky tofu, and delicate egg creates a comforting and satisfying main course. Top with fresh scallions for added flavor.',
 'This versatile California beef rub is easy to make and can be used for all types of beef dishes. Simply combine the ingredients and store it fo

In [29]:
recipe_df = pd.read_csv('../data/recipes.csv')

In [32]:
names = recipe_df['Name'].unique()

In [38]:
len([r for r in recipes_data.keys() if r not in names])

1058

In [53]:
print(len(recipes_data))
recipes_data = {k: v for k, v in recipes_data.items() if v.get('canonical_url') is not None}
print(len(recipes_data))

1261
1261


In [22]:
import random
recipes_with_descriptions = [recipe for recipe in recipes_data.values() if recipe.get('description') is not None]
recipes_without_descriptions = [recipe for recipe in recipes_data.values() if recipe.get('description') is None]
client = OpenAI()
def recipe_to_str(recipe):
        return f"""
    Title: {recipe.get('title')}
    category: {recipe.get('category')}
    instructions: {recipe.get('instructions')}
    """
    
def get_recipe_description(recipe, examples = []):
        messages = [
            {"role": "system", "content": "You are a foodwriter, adept at writing concise and helpful descriptions for recipes. Use the information given but also feel free to bring in your own cooking knowledge"},
            {"role": "system", "content": "Rules: Keep your responses to less than 200 characters. Do not repeat the name of the dish. Do not sound too similar to the other examples."},
        ]
        for example in examples:
            messages += [
                {"role": "user", "content": "Write a brief description of the following dish:" + recipe_to_str(example)},
                {"role": "system", "content": example.get('description')}
            ]
        
        messages += [
            {"role": "user", "content": "Write a brief description of the following dish:" + recipe_to_str(recipe)},
        ]
        
        completion = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages=messages
        )
        
        return completion.choices[0].message.content
    
examples = recipes_with_descriptions
for recipe in recipes_without_descriptions:
    description = get_recipe_description(recipe, examples)
    recipes_data[recipe['title']]['description'] = description
    recipe['description'] = description
    examples.append(recipe)
    examples = random.sample(examples, 3)
    

In [23]:
examples

[{'author': 'Scott Loitsch',
  'canonical_url': 'https://cooking.nytimes.com/recipes/1023659-rice-krispie-treats',
  'category': 'easy, quick, snack, dessert',
  'host': 'cooking.nytimes.com',
  'image': 'https://static01.nyt.com/images/2022/12/12/multimedia/sl-rice-krispies-treats-1-a552/sl-rice-krispies-treats-1-a552-mediumSquareAt3X.jpg',
  'ingredients': ['1/2 cup/113 grams unsalted butter, preferably European-style, plus extra for greasing',
   '1 (12-ounce/340-gram) bag marshmallows (preferably standard size)',
   '1 1/4 teaspoons/5 grams kosher salt (such as Diamond Crystal)',
   '1 teaspoon vanilla extract',
   '6 cups/180 grams Rice Krispies cereal',
   'Rainbow sprinkles (optional, but highly recommended)',
   'Flaky salt (optional)'],
  'instructions': 'Butter a 9-inch square baking pan.\nIn a large pot, melt the butter over medium heat until it starts to foam, about 3 minutes. Reduce heat to low and continue to cook, stirring constantly with a silicone spatula, until the bu

In [14]:
get_recipe_description(recipes_without_descriptions[0])

ChatCompletionMessage(content='This grain bowl features roasted mushrooms, halloumi cheese, and a medley of raw vegetables on a bed of barley, topped with a tangy yogurt and olive sauce. A flavorful and satisfying meal or side dish.', role='assistant', function_call=None, tool_calls=None)

In [7]:
get_recipe_description(recipes_without_descriptions[0])

ChatCompletionMessage(content="This grain bowl features roasted mushrooms, halloumi, and a vibrant mix of raw vegetables. Topped with lemony barley and a yogurt-olive dressing, it's a delicious and filling meal.", role='assistant', function_call=None, tool_calls=None)

In [8]:
recipes_without_descriptions[0]

{'author': 'Ali Slagle',
 'canonical_url': 'https://cooking.nytimes.com/recipes/1023492-roasted-mushroom-and-halloumi-grain-bowl',
 'category': 'dinner, lunch, weeknight, main course, side dish',
 'host': 'cooking.nytimes.com',
 'image': 'https://static01.nyt.com/images/2022/09/20/dining/as-roasted-mushroom-grain-bowl/merlin_213230274_1d9c89ae-4388-4b4c-8db8-e47f24defbc3-mediumSquareAt3X.jpg',
 'ingredients': ['1 pound cremini or button mushrooms, sliced 1/4-inch thick',
  '1 bunch scallions, trimmed and cut into 1-inch lengths',
  '8 to 9 ounces halloumi, torn into bite-size pieces',
  '4 tablespoons extra-virgin olive oil, plus more for serving',
  'Kosher salt (such as Diamond Crystal) and black pepper',
  '1 cup pearled barley or farro',
  '1 cup Greek yogurt or sour cream',
  '1/2 cup Kalamata olives, pitted and coarsely chopped',
  '1 lemon',
  '1 1/2 to 2 cups thinly sliced raw vegetables (such as brussels sprouts, celery, fennel, cucumbers and radishes)',
  '1/3 cup torn mint, 